In [1]:
import os
import shutil
import glob

from astroquery.mast import Observations

from astropy.io import fits
from astropy.table import Table

from stwcs import updatewcs

2022-06-27 11:26:30.482 xcodebuild[1582:21576] Requested but did not find extension point with identifier Xcode.IDEKit.ExtensionSentinelHostApplications for extension Xcode.DebuggerFoundation.AppExtensionHosts.watchOS of plug-in com.apple.dt.IDEWatchSupportCore
2022-06-27 11:26:30.482 xcodebuild[1582:21576] Requested but did not find extension point with identifier Xcode.IDEKit.ExtensionPointIdentifierToBundleIdentifier for extension Xcode.DebuggerFoundation.AppExtensionToBundleIdentifierMap.watchOS of plug-in com.apple.dt.IDEWatchSupportCore


In [6]:
obs_table = Observations.query_criteria(proposal_id=16231, obs_id='jec401010*')


dl_table = Observations.download_products(obs_table['obsid'],
                                          productSubGroupDescription=['RAW', 'ASN', 'SPT'],
                                          mrp_only=False)

In [7]:
for row in dl_table:
    oldfname = row['Local Path']
    newfname = os.path.basename(oldfname)
    os.rename(oldfname, newfname)
    
# Delete the mastDownload directory and all subdirectories it contains.
shutil.rmtree('mastDownload')

In [8]:
asn_file = fits.open('jec401010_asn.fits')
asn_file.info()

Filename: jec401010_asn.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  ASN           1 BinTableHDU     25   5R x 3C   [14A, 14A, L]   


In [9]:
asn_file = 'jec401010_asn.fits'
raw_files = glob.glob('*_raw.fits')

In [10]:
with fits.open(asn_file) as hdulist:
    hdulist.info()
    
with fits.open(raw_files[0]) as hdulist:
    hdulist.info()

Filename: jec401010_asn.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  ASN           1 BinTableHDU     25   5R x 3C   [14A, 14A, L]   
Filename: jec401noq_raw.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     176   ()      
  1  SCI           1 ImageHDU        85   (1024, 1024)   int16   
  2  ERR           1 ImageHDU        49   ()      
  3  DQ            1 ImageHDU        43   ()      


In [11]:
with fits.open(asn_file, mode='update') as asn_hdu:
    asn_tab = asn_hdu[1].data
    asn_tab = asn_tab[asn_tab['MEMTYPE'] == 'EXP-DTH']

In [12]:
cte_correct = False

In [34]:
for file in raw_files:
    
    if cte_correct: 
        value = 'PERFORM'
    else: 
        value = 'OMIT'
        
    fits.setval(file, 'PCTECORR', value=value)
    
    fits.setval(file, 'DARKCORR', value = 'OMIT')
    


In [14]:
os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = './crds_cache'
os.environ['jref'] = './crds_cache/references/hst/acs/'

In [15]:
for file in raw_files:
    command_line_input = 'crds bestrefs --files {:} --sync-references=1 --update-bestrefs'.format(file)
    os.system(command_line_input)

CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_wf4tfile_0250.rmap       678 bytes  (1 / 140 files) (0 / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_shadfile_0250.rmap       977 bytes  (2 / 140 files) (678 / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_offtab_0250.rmap         642 bytes  (3 / 140 files) (1.7 K / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_maskfile_0250.rmap       685 bytes  (4 / 140 files) (2.3 K / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_idctab_0250.rmap         696 bytes  (5 / 140 files) (3.0 K / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_flatfile_0250.rmap    30.0 K bytes  (6 / 140 files) (3.7 K / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2_dgeofile_0250.rmap       801 bytes  (7 / 140 files) (33.7 K / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_wfpc2

CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_exstab_0250.rmap          745 bytes  (61 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_echsctab_0250.rmap        749 bytes  (62 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_disptab_0250.rmap         813 bytes  (63 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_darkfile_0329.rmap     53.7 K bytes  (64 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_crrejtab_0250.rmap        711 bytes  (65 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_cdstab_0250.rmap          745 bytes  (66 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_stis_ccdtab_0252.rmap          893 bytes  (67 / 140 files) (1.2 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/h

CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_npolfile_0253.rmap       3.2 K bytes  (123 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_mlintab_0250.rmap          646 bytes  (124 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_mdriztab_0253.rmap         769 bytes  (125 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_imphttab_0258.rmap         769 bytes  (126 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_idctab_0256.rmap         1.5 K bytes  (127 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_flshfile_0266.rmap       3.3 K bytes  (128 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/mappings/hst/hst_acs_drkcfile_0415.rmap      16.4 K bytes  (129 / 140 files) (1.5 M / 1.7 M bytes)
CRDS - INFO -  Fetching  ./crds_cache/map

In [35]:
os.system('calacs.e jec401010_asn.fits');

git tag: 2.7.1
git branch: HEAD
HEAD @: a1d9cd3f61883e6115c1bab04cdf072946d1658a


CALBEG*** CALACS -- Version 10.3.2 (26-Apr-2021) ***
Begin    27-Jun-2022 12:25:33 CDT


Input    jec401010_asn.fits
LoadAsn:  Processing FULL Association
Trying to open jec401010_asn.fits...
Read in Primary header from jec401010_asn.fits...
Starting to process MAMA data now...
Warning    FLSHCORR keyword not found...
Revising existing trailer file `jec401noq.tra'.


CALBEG*** ACS2D -- Version 10.3.2 (26-Apr-2021) ***
Begin    27-Jun-2022 12:25:33 CDT
Input    jec401noq_raw.fits
Output   jec401noq_flt.fits
Warning    Output file `jec401noq_flt.fits' already exists.
ERROR:    Couldn't process MAMA data
ERROR:    CALACS processing NOT completed for jec401010_asn.fits
ERROR:    CALACS processing NOT completed for jec401010_asn.fits
ERROR:             status = 1021


In [32]:
if cte_correct:
    img_files = 'jec401*n[o-x]q_flc.fits'
else:
    img_files = 'jec401*n[o-x]q_flt.fits'

updatewcs.updatewcs(img_files, use_db=False)

- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A
- IDCTAB: Distortion model from row 5 for chip 1 : F150LP,N/A


['jec401nvq_flt.fits',
 'jec401noq_flt.fits',
 'jec401nrq_flt.fits',
 'jec401nxq_flt.fits']